In [52]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import csv

In [53]:
# Load training and testing data
training_data = pd.read_csv("train.csv")
testing_data = pd.read_csv("test.csv")

# Store the unique identifier of each passenger in a separate dataframe for later use
trainingIndex = pd.DataFrame()
trainingIndex["PassengerId"] = training_data["PassengerId"]

testingIndex = pd.DataFrame()
testingIndex["PassengerId"] = testing_data["PassengerId"]

# Extract target values
trainingTargets = pd.DataFrame()
trainingTargets["Survived"] = training_data["Survived"]

# Remove columns containing information that is not strictly related to a passenger's survival
columns_to_drop = ["PassengerId", "Name", "Ticket", "Fare", "Embarked", "Survived"]
training_data = training_data.drop(columns=columns_to_drop, axis=1)
testing_data = testing_data.drop(columns=list(set(columns_to_drop) - {"Survived"}), axis=1)

In [54]:
# Find non numerical columns
columns_to_encode = training_data.select_dtypes(exclude=['number']).columns

# One hot encode the 'Sex' column
training_data = pd.get_dummies(training_data, columns=['Sex'])
testing_data = pd.get_dummies(testing_data, columns=['Sex'])

In [55]:
# Convert cabin numbers to simple cabin classes. E.g C58 --> C, A12 --> A
def extract_first_char(value):
    if pd.notna(value): 
        return value[0] 
    else:
        return np.nan

training_data['Cabin'] = training_data['Cabin'].apply(lambda x: extract_first_char(x))
testing_data['Cabin'] = testing_data['Cabin'].apply(lambda x: extract_first_char(x))

In [56]:
# Columns to impute
mean_imputation_columns = ["Age"]
most_frequent_imputation_columns = ["Cabin"]

# Instantiate the imputers
mean_imputer = SimpleImputer(strategy='mean')
most_frequent_imputer = SimpleImputer(strategy='most_frequent')

# Fit the imputer on the selected columns
mean_imputer.fit(training_data[mean_imputation_columns])
most_frequent_imputer.fit(training_data[most_frequent_imputation_columns])

# Transform the selected columns
training_data[mean_imputation_columns] = mean_imputer.transform(training_data[mean_imputation_columns])
training_data[most_frequent_imputation_columns] = most_frequent_imputer.transform(training_data[most_frequent_imputation_columns])

testing_data[mean_imputation_columns] = mean_imputer.transform(testing_data[mean_imputation_columns])
testing_data[most_frequent_imputation_columns] = most_frequent_imputer.transform(testing_data[most_frequent_imputation_columns])

# One hot encode the 'Cabin' column
training_data = pd.get_dummies(training_data, columns=['Cabin'])
testing_data = pd.get_dummies(testing_data, columns=['Cabin'])

In [57]:
# Scale age values to the 0-1 value range
scaler = MinMaxScaler()
training_data["Age"] = scaler.fit_transform(training_data[["Age"]])
testing_data["Age"] = scaler.transform(testing_data[["Age"]])

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': np.arange(0,101,5)
}

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=10, scoring='accuracy')

# Fit GridSearchCV to training data
grid_search.fit(training_data, trainingTargets)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f'Best parameters: {best_params}')
print(f'Best cross-validation accuracy: {best_score:.2f}')

/Users/ioannisdrossas/Desktop/Titanic/venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ioannisdrossas/Desktop/Titanic/venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ioannisdrossas/Desktop/Titanic/venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/ioannisdrossas/Desktop/Titanic/venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionW

Best parameters: {'n_estimators': 60}
Best cross-validation accuracy: 0.81


/Users/ioannisdrossas/Desktop/Titanic/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
10 fits failed out of a total of 210.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/ioannisdrossas/Desktop/Titanic/venv/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ioannisdrossas/Desktop/Titanic/venv/lib/python3.10/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/Users/ioannisdrossas/Desktop/Titanic/venv/lib/python3.10/site-packages/sklearn/base.py", 

In [59]:
filename = 'my_predictions_random_forest.csv'
testing_data["Cabin_T"] = 0

rf_classifier = RandomForestClassifier(n_estimators=best_params['n_estimators'], random_state=42)
rf_classifier.fit(training_data, trainingTargets)
predictions_list = list(rf_classifier.predict(testing_data))
                        
index_list = testingIndex['PassengerId'].tolist()

with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    # Write header row
    csvwriter.writerow(["PassengerId", "Survived"])

    # Write data rows
    for index in range(418):
        passenger_id = index_list[index]
        prediction = predictions_list[index]
        csvwriter.writerow([passenger_id, prediction])


/Users/ioannisdrossas/Desktop/Titanic/venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
